# Conv1D

In [2]:
############ IMPORTS ####################
import sys
sys.path.append('./../')
import os
import numpy as np
from scipy import io
from sklearn.decomposition import PCA
from PIL import Image
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, cohen_kappa_score
import torch
from operator import truediv
import record
import keras.backend as K
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras.losses import categorical_crossentropy
from keras.layers import Activation, BatchNormalization, Conv1D, Dense, Flatten, MaxPooling1D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.utils import to_categorical as keras_to_categorical

############# CONFIGS ##########################
# os.environ["CUDA_VISIBLE_DEVICES"]="3"
datasetNames = ["Trento"]
patchsize1 = 11
patchsize2 = 11
batchsize = 256
EPOCH = 200
LR = 0.001


def AA_andEachClassAccuracy(confusion_matrix):
    counter = confusion_matrix.shape[0]
    list_diag = np.diag(confusion_matrix)
    list_raw_sum = np.sum(confusion_matrix, axis=1)
    each_acc = np.nan_to_num(truediv(list_diag, list_raw_sum))
    average_acc = np.mean(each_acc)
    return each_acc*100, average_acc*100

def get_model_compiled(bands, num_class):
    clf = Sequential()
    clf.add(Conv1D(20, (24), activation='relu', input_shape=(patchsize1*patchsize2,bands)))
    clf.add(MaxPooling1D(pool_size=5))
    clf.add(Flatten())
    clf.add(Dense(100))
    clf.add(BatchNormalization())
    clf.add(Activation('relu'))
    clf.add(Dense(num_class, activation='softmax'))
    clf.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=['accuracy'])
    return clf
        
for datasetName in datasetNames:
    print("----------------------------------Training for ",datasetName,"---------------------------------------------")
    
    try:
        os.makedirs(datasetName)
    except FileExistsError:
        pass
    # Train data
    HSI = io.loadmat('./../'+datasetName+'11x11/HSI_Tr.mat')
    TrainPatch = HSI['Data']
    TrainPatch = TrainPatch.astype(np.float32)
    NC = TrainPatch.shape[3] # NC is number of bands
    
    label = io.loadmat('./../'+datasetName+'11x11/TrLabel.mat')
    TrLabel = label['Data']
    
    # Test data
    HSI = io.loadmat('./../'+datasetName+'11x11/HSI_Te.mat')
    TestPatch = HSI['Data']
    TestPatch = TestPatch.astype(np.float32)
    
    label = io.loadmat('./../'+datasetName+'11x11/TeLabel.mat')
    TsLabel = label['Data']

    
    TrainPatch1 = torch.from_numpy(TrainPatch)
#         TrainPatch1 = TrainPatch1.permute(0,3,1,2)
    TrainLabel1 = torch.from_numpy(TrLabel)-1
    TrainLabel1 = TrainLabel1.long()
    

    TestPatch1 = torch.from_numpy(TestPatch)
#         TestPatch1 = TestPatch1.permute(0,3,1,2)
    TestLabel1 = torch.from_numpy(TsLabel)-1
    TestLabel1 = TestLabel1.long()

    Classes = len(np.unique(TrainLabel1))
    print("Train data shape = ", TrainPatch1.shape)
    print("Train label shape = ", TrainLabel1.shape)
    print("Test data shape = ", TestPatch1.shape)
    print("Test label shape = ", TestLabel1.shape)
    
    KAPPA = []
    OA = []
    AA = []
    ELEMENT_ACC = np.zeros((3, Classes))
    
    for iter in range(3):
        clf = get_model_compiled(NC, Classes)
        clf.summary()
        valdata = (TestPatch1.reshape(TestPatch1.shape[0],TestPatch1.shape[1]*TestPatch1.shape[2],TestPatch1.shape[3]).cpu().detach().numpy(), keras_to_categorical(TestLabel1.reshape(-1).cpu().detach().numpy(),Classes))
        clf.fit(TrainPatch1.reshape(TrainPatch1.shape[0],TrainPatch1.shape[1]*TrainPatch1.shape[2],TrainPatch1.shape[3]).cpu().detach().numpy(), keras_to_categorical(TrainLabel1.reshape(-1).cpu().detach().numpy(),Classes),
                            batch_size=batchsize,
                            epochs=EPOCH,
                            verbose=True,
                            validation_data=valdata,
                            callbacks = [ModelCheckpoint(datasetName+"/best_model_HSIOnly_Conv1D.h5", monitor='val_accuracy', verbose=0, save_best_only=True)])



        clf = load_model(datasetName+"/best_model_HSIOnly_Conv1D.h5")
        pred_y = np.argmax(clf.predict(TestPatch1.reshape(TestPatch1.shape[0],TestPatch1.shape[1]*TestPatch1.shape[2],TestPatch1.shape[3]).cpu().detach().numpy()), axis=1)

        y_test = TestLabel1.reshape(-1).cpu().detach().numpy()
        oa = accuracy_score(y_test, pred_y)*100
        confusion = confusion_matrix(y_test, pred_y)
        each_acc, aa = AA_andEachClassAccuracy(confusion)
        kappa = cohen_kappa_score(y_test, pred_y)*100
        KAPPA.append(kappa)
        OA.append(oa)
        AA.append(aa)
        ELEMENT_ACC[iter, :] = each_acc

    print("--------" + datasetName + " Training Finished-----------")
    record.record_output(OA, AA, KAPPA, ELEMENT_ACC,'./' + datasetName +'/Conv1D_Report_' + datasetName +'.txt')

    

----------------------------------Training for  Trento ---------------------------------------------
Train data shape =  torch.Size([819, 11, 11, 63])
Train label shape =  torch.Size([1, 819])
Test data shape =  torch.Size([29395, 11, 11, 63])
Test label shape =  torch.Size([1, 29395])
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 98, 20)            30260     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 19, 20)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 380)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               38100     
_________________________________________________________________
batch_normalization_1 (Batch (N

Epoch 48/200
4/4 [==============================] - 0s 108ms/step - loss: 0.0054 - accuracy: 1.0000 - val_loss: 0.3218 - val_accuracy: 0.9080
Epoch 49/200
1/4 [======>.......................] - ETA: 0s - loss: 0.0052 - accuracy: 1.0000

KeyboardInterrupt: 

# Conv2D

In [8]:
############ IMPORTS ####################
import sys
sys.path.append('./../')
import os
import numpy as np
from scipy import io
from sklearn.decomposition import PCA
from PIL import Image
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, cohen_kappa_score
import torch
from operator import truediv
import record
import keras.backend as K
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras.losses import categorical_crossentropy
from keras.layers import Activation, BatchNormalization, Conv2D, Dense, Flatten, MaxPooling2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras import regularizers
from keras.utils import to_categorical as keras_to_categorical

############# CONFIGS ##########################
# os.environ["CUDA_VISIBLE_DEVICES"]="3"
datasetNames = ["Trento"]
batchsize = 256
EPOCH = 200
LR = 0.001


def AA_andEachClassAccuracy(confusion_matrix):
    counter = confusion_matrix.shape[0]
    list_diag = np.diag(confusion_matrix)
    list_raw_sum = np.sum(confusion_matrix, axis=1)
    each_acc = np.nan_to_num(truediv(list_diag, list_raw_sum))
    average_acc = np.mean(each_acc)
    return each_acc*100, average_acc*100

def get_model_compiled(shapeinput, num_class, w_decay=0):
    clf = Sequential()
    clf.add(Conv2D(50, kernel_size=(5, 5), input_shape=shapeinput))
    clf.add(Activation('relu'))
    clf.add(Conv2D(100, (5, 5)))
    clf.add(Activation('relu'))
    clf.add(MaxPooling2D(pool_size=(2, 2)))
    clf.add(Flatten())
    clf.add(Dense(100, kernel_regularizer=regularizers.l2(w_decay)))
    clf.add(Activation('relu'))
    clf.add(Dense(num_class, activation='softmax'))
    clf.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=['accuracy'])
    return clf

        
for datasetName in datasetNames:
    print("----------------------------------Training for ",datasetName,"---------------------------------------------")
    
    try:
        os.makedirs(datasetName)
    except FileExistsError:
        pass
    
    # Train data
    HSI = io.loadmat('./../'+datasetName+'11x11/HSI_Tr.mat')
    TrainPatch = HSI['Data']
    TrainPatch = TrainPatch.astype(np.float32)
    NC = TrainPatch.shape[3] # NC is number of bands
    
    label = io.loadmat('./../'+datasetName+'11x11/TrLabel.mat')
    TrLabel = label['Data']
    
    # Test data
    HSI = io.loadmat('./../'+datasetName+'11x11/HSI_Te.mat')
    TestPatch = HSI['Data']
    TestPatch = TestPatch.astype(np.float32)
    
    label = io.loadmat('./../'+datasetName+'11x11/TeLabel.mat')
    TsLabel = label['Data']

    
    TrainPatch1 = torch.from_numpy(TrainPatch)
#         TrainPatch1 = TrainPatch1.permute(0,3,1,2)
    TrainLabel1 = torch.from_numpy(TrLabel)-1
    TrainLabel1 = TrainLabel1.long()
    

    TestPatch1 = torch.from_numpy(TestPatch)
#         TestPatch1 = TestPatch1.permute(0,3,1,2)
    TestLabel1 = torch.from_numpy(TsLabel)-1
    TestLabel1 = TestLabel1.long()

    Classes = len(np.unique(TrainLabel1))
    print("Train data shape = ", TrainPatch1.shape)
    print("Train label shape = ", TrainLabel1.shape)
    print("Test data shape = ", TestPatch1.shape)
    print("Test label shape = ", TestLabel1.shape)
    
    KAPPA = []
    OA = []
    AA = []
    ELEMENT_ACC = np.zeros((3, Classes))
    
    for iter in range(3):
        inputshape = TrainPatch1.shape[1:]
        clf = get_model_compiled(inputshape, Classes, w_decay = 0.02)
        clf.summary()
        valdata = (TestPatch1.cpu().detach().numpy(), keras_to_categorical(TestLabel1.reshape(-1).cpu().detach().numpy(),Classes))
        clf.fit(TrainPatch1.cpu().detach().numpy(), keras_to_categorical(TrainLabel1.reshape(-1).cpu().detach().numpy(),Classes),
                            batch_size=batchsize,
                            epochs=EPOCH,
                            verbose=True,
                            validation_data=valdata,
                            callbacks = [ModelCheckpoint(datasetName+"/best_model_HSIOnly_Conv2D.h5", monitor='val_accuracy', verbose=0, save_best_only=True)])



        clf = load_model(datasetName+"/best_model_HSIOnly_Conv2D.h5")
        pred_y = np.argmax(clf.predict(TestPatch1.cpu().detach().numpy()), axis=1)

        y_test = TestLabel1.reshape(-1).cpu().detach().numpy()
        oa = accuracy_score(y_test, pred_y)*100
        confusion = confusion_matrix(y_test, pred_y)
        each_acc, aa = AA_andEachClassAccuracy(confusion)
        kappa = cohen_kappa_score(y_test, pred_y)*100
        KAPPA.append(kappa)
        OA.append(oa)
        AA.append(aa)
        ELEMENT_ACC[iter, :] = each_acc

    print("--------" + datasetName + " Training Finished-----------")
    record.record_output(OA, AA, KAPPA, ELEMENT_ACC,'./' + datasetName +'/Conv2D_Report_' + datasetName +'.txt')

    

----------------------------------Training for  Trento ---------------------------------------------
Train data shape =  torch.Size([819, 11, 11, 63])
Train label shape =  torch.Size([1, 819])
Test data shape =  torch.Size([29395, 11, 11, 63])
Test label shape =  torch.Size([1, 29395])
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 7, 7, 50)          78800     
_________________________________________________________________
activation_7 (Activation)    (None, 7, 7, 50)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 3, 3, 100)         125100    
_________________________________________________________________
activation_8 (Activation)    (None, 3, 3, 100)         0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (N

KeyboardInterrupt: 

# Conv3D

In [4]:
############ IMPORTS ####################
import os
import numpy as np
from scipy import io
from sklearn.decomposition import PCA
from PIL import Image
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, cohen_kappa_score
import torch
from operator import truediv
import record
import keras.backend as K
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras.losses import categorical_crossentropy
from keras.layers import Activation, BatchNormalization, Conv3D, Dense, Flatten, MaxPooling3D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras import regularizers
from keras.utils import to_categorical as keras_to_categorical


############# CONFIGS ##########################
# os.environ["CUDA_VISIBLE_DEVICES"]="2"
datasetNames = ["Trento"]
batchsize = 256
EPOCH = 200
LR = 0.001


def AA_andEachClassAccuracy(confusion_matrix):
    counter = confusion_matrix.shape[0]
    list_diag = np.diag(confusion_matrix)
    list_raw_sum = np.sum(confusion_matrix, axis=1)
    each_acc = np.nan_to_num(truediv(list_diag, list_raw_sum))
    average_acc = np.mean(each_acc)
    return each_acc*100, average_acc*100

def get_model_compiled(shapeinput, num_class, w_decay=0, lr=1e-3):
            clf = Sequential()
            clf.add(Conv3D(32, kernel_size=(5, 5, 24), input_shape=shapeinput))
            clf.add(BatchNormalization())
            clf.add(Activation('relu'))
            clf.add(Conv3D(64, (5, 5, 16)))
            clf.add(BatchNormalization())
            clf.add(Activation('relu'))
            clf.add(MaxPooling3D(pool_size=(2, 2, 1)))
            clf.add(Flatten())
            clf.add(Dense(300, kernel_regularizer=regularizers.l2(w_decay)))
            clf.add(BatchNormalization())
            clf.add(Activation('relu'))
            clf.add(Dense(num_class, activation='softmax'))
            clf.compile(loss=categorical_crossentropy, optimizer=Adam(lr=lr), metrics=['accuracy'])
            return clf
        
        
for datasetName in datasetNames:
    print("----------------------------------Training for ",datasetName,"---------------------------------------------")
    
    try:
        os.makedirs(datasetName)
    except FileExistsError:
        pass
    
    # Train data
    HSI = io.loadmat('./../'+datasetName+'11x11/HSI_Tr.mat')
    TrainPatch = HSI['Data']
    TrainPatch = TrainPatch.astype(np.float32)
    NC = TrainPatch.shape[3] # NC is number of bands
    
    label = io.loadmat('./../'+datasetName+'11x11/TrLabel.mat')
    TrLabel = label['Data']
    
    # Test data
    HSI = io.loadmat('./../'+datasetName+'11x11/HSI_Te.mat')
    TestPatch = HSI['Data']
    TestPatch = TestPatch.astype(np.float32)
    
    label = io.loadmat('./../'+datasetName+'11x11/TeLabel.mat')
    TsLabel = label['Data']

    
    TrainPatch1 = torch.from_numpy(TrainPatch)
#         TrainPatch1 = TrainPatch1.permute(0,3,1,2)
    TrainLabel1 = torch.from_numpy(TrLabel)-1
    TrainLabel1 = TrainLabel1.long()
    

    TestPatch1 = torch.from_numpy(TestPatch)
#         TestPatch1 = TestPatch1.permute(0,3,1,2)
    TestLabel1 = torch.from_numpy(TsLabel)-1
    TestLabel1 = TestLabel1.long()

    Classes = len(np.unique(TrainLabel1))
    print("Train data shape = ", TrainPatch1.shape)
    print("Train label shape = ", TrainLabel1.shape)
    print("Test data shape = ", TestPatch1.shape)
    print("Test label shape = ", TestLabel1.shape)
    
    KAPPA = []
    OA = []
    AA = []
    ELEMENT_ACC = np.zeros((3, Classes))

    for iter in range(3):
        inputshape = TrainPatch1.shape[1:]
        clf = get_model_compiled(inputshape, Classes, w_decay = 0.02)
        clf.summary()
        valdata = (TestPatch1.cpu().detach().numpy(), keras_to_categorical(TestLabel1.reshape(-1).cpu().detach().numpy(),Classes))
        clf.fit(TrainPatch1.cpu().detach().numpy(), keras_to_categorical(TrainLabel1.reshape(-1).cpu().detach().numpy(),Classes),
                            batch_size=batchsize,
                            epochs=EPOCH,
                            verbose=True,
                            validation_data=valdata,
                            callbacks = [ModelCheckpoint(datasetName+"/best_model_HSIOnly_Conv3D.h5", monitor='val_accuracy', verbose=0, save_best_only=True)])



        clf = load_model(datasetName+"/best_model_HSIOnly_Conv3D.h5")
        pred_y = np.argmax(clf.predict(TestPatch1.cpu().detach().numpy()), axis=1)

        y_test = TestLabel1.reshape(-1).cpu().detach().numpy()
        oa = accuracy_score(y_test, pred_y)*100
        confusion = confusion_matrix(y_test, pred_y)
        each_acc, aa = AA_andEachClassAccuracy(confusion)
        kappa = cohen_kappa_score(y_test, pred_y)*100
        KAPPA.append(kappa)
        OA.append(oa)
        AA.append(aa)
        ELEMENT_ACC[iter, :] = each_acc

    print("--------" + datasetName + " Training Finished-----------")
    record.record_output(OA, AA, KAPPA, ELEMENT_ACC,'./' + datasetName +'/Conv3D_Report_' + datasetName +'.txt')

    

Trento
True
(819, 11, 11, 63)
Test patch 1 shape  torch.Size([29395, 11, 11, 63])
HSI Only
Epoch 1/200
4/4 [==============================] - 26s 7s/step - loss: 11.6432 - accuracy: 0.4520 - val_loss: 12.3483 - val_accuracy: 0.4354
Epoch 2/200
4/4 [==============================] - 22s 6s/step - loss: 9.7266 - accuracy: 0.9265 - val_loss: 14.8764 - val_accuracy: 0.4434
Epoch 3/200
4/4 [==============================] - 23s 6s/step - loss: 8.9178 - accuracy: 0.9397 - val_loss: 15.7710 - val_accuracy: 0.4366
Epoch 4/200
4/4 [==============================] - 22s 6s/step - loss: 8.0941 - accuracy: 0.9596 - val_loss: 14.9355 - val_accuracy: 0.4287
Epoch 5/200
4/4 [==============================] - 22s 6s/step - loss: 7.2770 - accuracy: 0.9845 - val_loss: 13.4622 - val_accuracy: 0.4247
Epoch 6/200
4/4 [==============================] - 23s 6s/step - loss: 6.5056 - accuracy: 0.9695 - val_loss: 11.9813 - val_accuracy: 0.4280
Epoch 7/200
4/4 [==============================] - 21s 6s/step - los

Epoch 59/200
4/4 [==============================] - 22s 6s/step - loss: 0.0900 - accuracy: 0.9977 - val_loss: 2.0978 - val_accuracy: 0.1544
Epoch 60/200
4/4 [==============================] - 22s 6s/step - loss: 0.0997 - accuracy: 0.9929 - val_loss: 3.2336 - val_accuracy: 0.1363
Epoch 61/200
4/4 [==============================] - 22s 6s/step - loss: 0.0800 - accuracy: 0.9995 - val_loss: 4.1595 - val_accuracy: 0.1630
Epoch 62/200
4/4 [==============================] - 23s 6s/step - loss: 0.0795 - accuracy: 0.9989 - val_loss: 5.0591 - val_accuracy: 0.1549
Epoch 63/200
4/4 [==============================] - 23s 6s/step - loss: 0.0924 - accuracy: 0.9968 - val_loss: 4.2626 - val_accuracy: 0.1566
Epoch 64/200
4/4 [==============================] - 22s 6s/step - loss: 0.1319 - accuracy: 0.9692 - val_loss: 3.0407 - val_accuracy: 0.2962
Epoch 65/200
4/4 [==============================] - 22s 6s/step - loss: 0.1038 - accuracy: 0.9980 - val_loss: 2.4460 - val_accuracy: 0.4066
Epoch 66/200
4/4 [==

4/4 [==============================] - 23s 6s/step - loss: 0.0239 - accuracy: 1.0000 - val_loss: 1.8544 - val_accuracy: 0.4379
Epoch 176/200
4/4 [==============================] - 22s 6s/step - loss: 0.0192 - accuracy: 1.0000 - val_loss: 1.7894 - val_accuracy: 0.3425
Epoch 177/200
4/4 [==============================] - 22s 6s/step - loss: 0.0156 - accuracy: 1.0000 - val_loss: 1.8309 - val_accuracy: 0.2174
Epoch 178/200
4/4 [==============================] - 22s 6s/step - loss: 0.0126 - accuracy: 1.0000 - val_loss: 1.9423 - val_accuracy: 0.1438
Epoch 179/200
4/4 [==============================] - 23s 6s/step - loss: 0.0103 - accuracy: 1.0000 - val_loss: 2.1088 - val_accuracy: 0.1129
Epoch 180/200
4/4 [==============================] - 22s 6s/step - loss: 0.0086 - accuracy: 1.0000 - val_loss: 1.8970 - val_accuracy: 0.2023
Epoch 181/200
4/4 [==============================] - 22s 6s/step - loss: 0.0092 - accuracy: 0.9995 - val_loss: 4.9268 - val_accuracy: 0.1346
Epoch 182/200
4/4 [========

4/4 [==============================] - 22s 6s/step - loss: 0.0067 - accuracy: 1.0000 - val_loss: 3.7238 - val_accuracy: 0.1465
Epoch 92/200
4/4 [==============================] - 22s 6s/step - loss: 0.0099 - accuracy: 1.0000 - val_loss: 2.9289 - val_accuracy: 0.2284
Epoch 93/200
4/4 [==============================] - 22s 6s/step - loss: 0.0117 - accuracy: 1.0000 - val_loss: 1.8211 - val_accuracy: 0.4075
Epoch 94/200
4/4 [==============================] - 22s 6s/step - loss: 0.0234 - accuracy: 0.9955 - val_loss: 4.8156 - val_accuracy: 0.1844
Epoch 95/200
4/4 [==============================] - 22s 6s/step - loss: 0.2822 - accuracy: 0.9457 - val_loss: 2.5518 - val_accuracy: 0.2769
Epoch 96/200
4/4 [==============================] - 22s 6s/step - loss: 0.0794 - accuracy: 0.9841 - val_loss: 2.1089 - val_accuracy: 0.5564
Epoch 97/200
4/4 [==============================] - 22s 6s/step - loss: 0.1239 - accuracy: 0.9778 - val_loss: 1.7274 - val_accuracy: 0.5287
Epoch 98/200
4/4 [===============

4/4 [==============================] - 22s 6s/step - loss: 5.7743 - accuracy: 0.9906 - val_loss: 12.4807 - val_accuracy: 0.3919
Epoch 8/200
4/4 [==============================] - 22s 6s/step - loss: 5.0931 - accuracy: 0.9935 - val_loss: 10.5363 - val_accuracy: 0.4104
Epoch 9/200
4/4 [==============================] - 23s 6s/step - loss: 4.4716 - accuracy: 0.9985 - val_loss: 9.0545 - val_accuracy: 0.4285
Epoch 10/200
4/4 [==============================] - 22s 6s/step - loss: 3.9160 - accuracy: 0.9943 - val_loss: 7.7186 - val_accuracy: 0.4505
Epoch 11/200
4/4 [==============================] - 22s 6s/step - loss: 3.4131 - accuracy: 0.9981 - val_loss: 6.5944 - val_accuracy: 0.4620
Epoch 12/200
4/4 [==============================] - 22s 6s/step - loss: 2.9772 - accuracy: 0.9966 - val_loss: 5.6014 - val_accuracy: 0.4932
Epoch 13/200
4/4 [==============================] - 22s 6s/step - loss: 2.5788 - accuracy: 1.0000 - val_loss: 4.7288 - val_accuracy: 0.4975
Epoch 14/200
4/4 [===============

4/4 [==============================] - 23s 6s/step - loss: 0.0351 - accuracy: 1.0000 - val_loss: 0.9286 - val_accuracy: 0.7089
Epoch 124/200
4/4 [==============================] - 22s 6s/step - loss: 0.0349 - accuracy: 1.0000 - val_loss: 0.9592 - val_accuracy: 0.6935
Epoch 125/200
4/4 [==============================] - 23s 6s/step - loss: 0.0320 - accuracy: 1.0000 - val_loss: 0.8196 - val_accuracy: 0.7258
Epoch 126/200
4/4 [==============================] - 22s 6s/step - loss: 0.0283 - accuracy: 1.0000 - val_loss: 0.5479 - val_accuracy: 0.8960
Epoch 127/200
4/4 [==============================] - 22s 6s/step - loss: 0.0264 - accuracy: 1.0000 - val_loss: 0.4978 - val_accuracy: 0.9285
Epoch 128/200
4/4 [==============================] - 22s 6s/step - loss: 0.0205 - accuracy: 1.0000 - val_loss: 0.5473 - val_accuracy: 0.9162
Epoch 129/200
4/4 [==============================] - 22s 6s/step - loss: 0.0170 - accuracy: 1.0000 - val_loss: 0.7569 - val_accuracy: 0.8093
Epoch 130/200
4/4 [========

4


ValueError: Negative dimension size caused by subtracting 5 from 1 for '{{node conv3d_2/Conv3D}} = Conv3D[T=DT_FLOAT, data_format="NDHWC", dilations=[1, 1, 1, 1, 1], padding="VALID", strides=[1, 1, 1, 1, 1]](conv3d_2_input, conv3d_2/Conv3D/ReadVariableOp)' with input shapes: [?,1,176,11,11], [5,5,24,11,32].